# Kalman filter

In [7]:
!pip install pykalman

In [3]:
import numpy as np
import pykalman
from pykalman import KalmanFilter
from new_dataloading import X_testD, X_test_obsD,Obs_noise,Model_noise, mask_trainD,  Database
import torch
Obs_noise

... Data type: _ObsSubRnd_00_00
..... Training dataset: 600x10x30
..... Validation dataset: 200x10x30
..... Test dataset    : 200x10x30
........ Initialize interpolated states
(1000, 10, 30)
<class 'netCDF4._netCDF4.Variable'>
..... Training dataset: 600x10x30
..... Validation dataset: 200x10x30
..... Test dataset    : 200x10x30
<class 'numpy.ndarray'>
float64


0.001

In [2]:
print(0.8418+0.9337+0.9863+0.6442+0.7820)

D = Database[int(0.8*1000):,:,:]
print(D[0,0:5,0].shape)
indt = 8
indx = 5
print((torch.sum(D[0,(indx-1):(indx+2),indt-1])/4)+D[0,indx-2,indt-2]/4+D[0,indx+2,indt-2]/4)
print(D[0,indx,indt])
print(D[0,0:10,:])



4.188000000000001
torch.Size([5])
tensor(1.0524)
tensor(0.9419)
tensor([[0.4686, 0.6793, 0.4993, 0.3048, 0.1889, 0.1085, 0.0637, 0.0406, 0.0243,
         0.0155],
        [0.3135, 0.5599, 0.7284, 0.6453, 0.4786, 0.3385, 0.2369, 0.1787, 0.1219,
         0.0855],
        [0.3294, 0.5454, 0.6190, 0.6982, 0.7211, 0.6318, 0.5159, 0.4305, 0.3320,
         0.2448],
        [0.8812, 0.7774, 0.7281, 0.6861, 0.7114, 0.7613, 0.7191, 0.6913, 0.6362,
         0.5046],
        [0.2006, 0.6754, 0.8073, 0.7938, 0.7605, 0.7648, 0.8135, 0.7686, 0.7860,
         0.7332],
        [0.5250, 0.6733, 0.7091, 0.7900, 0.7777, 0.8235, 0.8795, 0.8620, 0.9419,
         0.8292],
        [0.9117, 0.9196, 0.8770, 0.8282, 0.8515, 0.8406, 0.9182, 0.9648, 0.9104,
         0.9035],
        [0.4775, 0.6918, 0.7944, 0.8392, 0.8656, 0.8949, 0.8949, 0.9418, 0.9697,
         1.0160],
        [0.2181, 0.4866, 0.7674, 0.8125, 0.8953, 0.9255, 0.9981, 1.0544, 1.0865,
         1.0966],
        [0.0389, 0.3588, 0.4505, 0.6199, 0.78

In [3]:
print(X_testD.shape)
t = X_testD.shape[1]
x = X_testD.shape[2]
transition_matrices = np.zeros((t-2,2*x,2*x))
A1 = 0.25*(np.eye(x)+np.eye(x, k=-1))
A1[x-1,0]=0.25
transition_matrices[0,:x,:x]= A1
transition_matrices[0,:x,x:]=0.25*(np.eye(x, k=2)+np.eye(x,k=-2)+np.eye(x,k=x-2)+np.eye(x,k=-x+2))
transition_matrices[0,:x,x:]= np.eye(x)
mod = 'fsum1'

if mod == 'fsum1':
    for i in range(1,t-2):
        transition_matrices[i,:x,:x] = A1
        transition_matrices[i,x:,:x] = np.eye(x)
        transition_matrices[i,:x,x:] = 0.25*(np.eye(x,k=-2)+np.eye(x,k=-x+2))
        

        
observation_matrix = np.identity(2*x)

transition_covariance = np.zeros((2*x,2*x))
transition_covariance[:x,:x] = Model_noise**2 * np.identity(x)

observation_covariance = np.zeros((2*x,2*x))
observation_covariance[:x,:x] = Obs_noise**2 * np.identity(x)

x1 = X_test_obsD[0,0,:]
x2 = X_test_obsD[0,1,:]   

Initial_state = np.zeros((2*x))
Initial_state[:x] = x2
Initial_state[x:] = x1
        
Initial_covariance = Model_noise**2 * np.identity(2*x)

torch.Size([200, 10, 30])


In [4]:
k = KalmanFilter(
    transition_matrices=transition_matrices, 
             observation_matrices=observation_matrix, 
             transition_covariance=transition_covariance, 
             observation_covariance=observation_covariance, 
             transition_offsets=None, 
             observation_offsets=None, 
             initial_state_mean=Initial_state, 
             initial_state_covariance=Initial_covariance, 
             random_state=None, 
             em_vars=['transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance'], 
             n_dim_state=None, 
             n_dim_obs=None
)

In [5]:

X_Filter = np.zeros((t-1,2*x))

for i in range(t-1):
    X_Filter[i,:x] = X_test_obsD[0,i+1,:]
    X_Filter[i,x:] = X_test_obsD[0,i,:]

mask = np.zeros((t-1,2*x))

mask[7:,:]+=1
mask[6,:x]+=1

    
X_train_mask = np.ma.masked_array(X_Filter,mask)

KF = k.filter(X_Filter)
print(np.diagonal(KF[1],axis1=1,axis2=2).shape)
print(X_test_obsD[0,1,:])
print(X_testD[0,1,:])


(9, 60)
tensor([0.6780, 0.5603, 0.5442, 0.7782, 0.6756, 0.6729, 0.9181, 0.6915, 0.4890,
        0.3599, 0.6827, 0.6458, 0.4155, 0.4203, 0.4655, 0.5436, 0.8167, 0.8046,
        0.6285, 0.5085, 0.5341, 0.6436, 0.7613, 0.9075, 0.8181, 0.8828, 0.9832,
        0.8127, 0.8406, 0.8440])
tensor([0.6793, 0.5599, 0.5454, 0.7774, 0.6754, 0.6733, 0.9196, 0.6918, 0.4866,
        0.3588, 0.6821, 0.6454, 0.4176, 0.4198, 0.4639, 0.5456, 0.8168, 0.8038,
        0.6312, 0.5083, 0.5350, 0.6438, 0.7619, 0.9073, 0.8186, 0.8829, 0.9848,
        0.8129, 0.8391, 0.8432])


# Prediction with Kalman FIlter for additive noise model

In [6]:
X_preds_KF = np.zeros(X_test_obsD.shape)
X_preds_KF[:,0:2,:] = X_test_obsD[:,0:2,:]
X_cov_KF = np.zeros((X_test_obsD.shape[0],t,x,x))
X_cov_KF[:,0,:] = Obs_noise**2*np.identity(x)
X_cov_KF[:,1,:] = Obs_noise**2*np.identity(x)


for i in range(X_test_obsD.shape[0]):    
    x1 = X_test_obsD[i,0,:]
    x2 = X_test_obsD[i,1,:]   

    Initial_state = np.zeros((2*x))
    Initial_state[:x] = x2
    Initial_state[x:] = x1
    k = KalmanFilter(
    transition_matrices=transition_matrices, 
             observation_matrices=observation_matrix, 
             transition_covariance=transition_covariance, 
             observation_covariance=observation_covariance, 
             transition_offsets=None, 
             observation_offsets=None, 
             initial_state_mean=Initial_state, 
             initial_state_covariance=Initial_covariance, 
             random_state=None, 
             em_vars=['transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance'], 
             n_dim_state=None, 
             n_dim_obs=None
)
    
    X_Filter = np.zeros((t-4,2*x))
    for j in range(t-4):
        X_Filter[j,:x] = X_test_obsD[i,j+1,:]
        X_Filter[j,x:] = X_test_obsD[i,j,:]
    KF = k.filter(X_Filter)
    X_preds_KF[i,2:7,:] = KF[0][1:,:x]
    X_cov_KF[i,2:7,:] = KF[1][1:,:x,:x]
    KFb = KF[1][-1,:,:]
    for j in range(7,t):
        KFb = np.dot(np.dot(transition_matrices[j-2,:,:],KFb),transition_matrices[j-2,:,:].T) +transition_covariance
        X_cov_KF[i,j,:,:] = KFb[:x,:x]
        X_preds_KF[i,j,:] = np.dot(transition_matrices[j-2,:x,:x],X_preds_KF[i,j-1,:]) + np.dot(transition_matrices[j-2,:x,x:],X_preds_KF[i,j-2,:])
    

In [7]:
ind=0
print(X_cov_KF[0,ind,:,:])

[[1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 1.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.

## Forecasts scores

In [8]:
def logarithmic_scores(target,mu,sig):
    return(0.5*(np.dot((np.dot((target-mu).T,np.linalg.inv(sig))),(target-mu)))+0.5*np.log(np.linalg.det(sig))+15*np.log(2*np.pi))


In [9]:
ind = 9
print(logarithmic_scores(X_testD[0,ind,:],X_preds_KF[0,ind,:],X_cov_KF[0,ind,:,:])/30)


274.5261868757858


# Forecasts with multiplicative noise model

In [1]:
t = X_testD.shape[1]
x = X_testD.shape[2]
transition_matrices = np.zeros((t-2,2*x,2*x))
A1 = 0.6*np.eye(x)+0.4*np.eye(x, k=-1)
A1[x-1,0]=0.2
transition_matrices[0,:x,:x]= A1
transition_matrices[0,:x,x:]=0.1*np.eye(x, k=2)+0.05*np.eye(x,k=-2)+0.1*np.eye(x,k=-x+2)+0.05*np.eye(x,k=x-2)
transition_matrices[0,x:,x:]= np.eye(x)

mod = 'fsum2'

if mod == 'fsum2':
    for i in range(1,t-2):
        transition_matrices[i,:x,:x] = A1
        transition_matrices[i,x:,:x] = np.eye(x)
        transition_matrices[i,:x,x:] = 0.05*np.eye(x,k=-2)+0.1*np.eye(x,k=-x+2)
        
        
observation_matrix = np.identity(2*x)

x1 = X_test_obsD[0,0,:]
x2 = X_test_obsD[0,1,:]   

Initial_state = np.zeros((2*x))
Initial_state[:x] = x2
Initial_state[x:] = x1
        
Initial_covariance = Model_noise**2 * np.identity(2*x)


observation_covariance = np.zeros((2*x,2*x))
observation_covariance[:x,:x] = Obs_noise**2 * np.identity(x)


NameError: name 'X_testD' is not defined

### Estimation of model error covariance matrix

In [2]:
Estimated_cov = np.zeros((t-2,x,x))

for i in range(t-2):
    A=np.zeros((D.shape[0],x,x))
    truth = D[:,:,i+2].detach().numpy()
    Estimated = np.dot(D[:,:,i+1],np.transpose(transition_matrices[i,:x,:x]))+np.dot(D[:,:,i],np.transpose(transition_matrices[i,:x,x:]))
    B=Estimated-truth
    B=np.reshape(B,(D.shape[0],x,1))
    for j in range(D.shape[0]):
        A[j,:,:]=np.dot(B[j,:],np.transpose(B[j,:]))
    Estimated_cov[i,:,:]=np.mean(A,axis=0)

NameError: name 'np' is not defined

In [48]:
print(Estimated_cov[6,:,:])

[[ 4.59528624e-07 -3.21446889e-07 -1.29747119e-06 -2.99497820e-06
  -6.22625372e-07  8.52624831e-07 -7.12889539e-07 -7.59294587e-07
   1.83139533e-06  1.64280930e-06 -7.49297455e-07 -2.56256303e-06
   1.13873216e-06  1.93355964e-06 -5.72249238e-07  3.86424834e-06
   3.52719210e-06  9.45496001e-07 -1.57284706e-06  1.18295186e-06
   5.81793125e-07 -2.18535777e-06 -6.07907987e-07 -7.43807494e-07
  -1.76006960e-06 -1.06802677e-06 -1.75413608e-06 -1.55532595e-06
   2.65152873e-07 -1.24936966e-06]
 [-3.21446889e-07  2.03772029e-05 -4.61563974e-06  5.35852799e-06
   1.50583571e-05 -2.21267589e-05  4.87554237e-05  9.28180819e-06
  -1.57453091e-05  4.80558175e-06 -1.58425709e-06 -4.89382643e-06
   1.30253799e-05 -2.91148056e-05 -3.06328633e-06  1.38452552e-05
  -1.09893605e-05  2.10997482e-05  1.67514498e-05 -1.47167340e-05
   1.25548034e-05 -3.09641190e-06  7.30747473e-06 -1.57494188e-05
   2.52663038e-05 -2.27280023e-05  7.24388429e-07 -1.64736885e-05
   7.03699614e-06  1.60311915e-05]
 [-1.2

In [49]:
X_preds_KF = np.zeros(X_test_obsD.shape)
X_preds_KF[:,0:2,:] = X_test_obsD[:,0:2,:]
X_cov_KF = np.zeros((X_test_obsD.shape[0],t,x,x))
X_cov_KF[:,0,:] = Obs_noise**2*np.identity(x)
X_cov_KF[:,1,:] = Obs_noise**2*np.identity(x)


for i in range(X_test_obsD.shape[0]): 
    for j in range(t-2):
        if j<(t-5):
            x1 = X_preds_KF[i,j,:]
            x2 = X_preds_KF[i,j+1,:]

            Initial_state = np.zeros((2*x))
            Initial_state[:x] = x2
            Initial_state[x:] = x1
            transition_covariance = np.zeros((2*x,2*x))
            transition_covariance[:x,:x] = Estimated_cov[j,:,:]
            k = KalmanFilter(
    transition_matrices=transition_matrices[j,:,:],
             observation_matrices=observation_matrix, 
             transition_covariance=transition_covariance,
             observation_covariance=observation_covariance, 
             transition_offsets=None, 
             observation_offsets=None, 
             initial_state_mean=Initial_state, 
             initial_state_covariance=Initial_covariance, 
             random_state=None, 
             em_vars=['transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance'], 
             n_dim_state=None, 
             n_dim_obs=None
)
    
            X_Filter = np.zeros((2,2*x))
            X_Filter[0,:x] = X_test_obsD[i,j+1,:]
            X_Filter[0,x:] = X_test_obsD[i,j,:]
            X_Filter[1,:x] = X_test_obsD[i,j+2,:]
            X_Filter[1,x:] = X_test_obsD[i,j+1,:]
            KF = k.filter(X_Filter)
            X_preds_KF[i,j+2,:] = KF[0][1,:x]
            X_cov_KF[i,j+2,:,:] = KF[1][1,:x,:x]
            
        else:
            KFb =  np.zeros((2*x,2*x))
            KFb[:x,:x] = X_cov_KF[i,j+1,:,:]
            Stimated = np.zeros((2*x,2*x))
            Stimated[:x,:x] = Estimated_cov[j,:,:]
            KFb = np.dot(np.dot(transition_matrices[j,:,:],KFb),transition_matrices[j,:,:].T) +Stimated
            X_cov_KF[i,j+2,:,:] = KFb[:x,:x]
            X_preds_KF[i,j+2,:] = np.dot(transition_matrices[j,:x,:x],X_preds_KF[i,j+1,:]) + np.dot(transition_matrices[j,:x,x:],X_preds_KF[i,j,:])
        if i ==0:
            print(X_preds_KF[0,:,:])

[[0.47039285 0.31397888 0.33033773 0.88006657 0.19930974 0.52690524
  0.91312802 0.47796354 0.21770976 0.03911652 0.9875167  0.31059328
  0.25546011 0.37109774 0.2013388  0.41265529 0.85447824 0.45743269
  0.49674678 0.29903078 0.38347036 0.49212116 0.57466632 0.79626751
  0.391776   0.91138154 0.67158121 0.50108153 0.71013618 0.642326  ]
 [0.67798328 0.56026053 0.54419088 0.77817339 0.67564213 0.67294264
  0.91807133 0.69145256 0.48902252 0.35986903 0.68269777 0.64584172
  0.41547269 0.420344   0.46545774 0.54359812 0.81668431 0.80456418
  0.62851095 0.50846225 0.5341109  0.6435535  0.76127911 0.90752345
  0.81810981 0.88276058 0.98322695 0.8127324  0.84059304 0.84401351]
 [0.49908103 0.72950733 0.62050091 0.72939923 0.80641584 0.70816777
  0.87599858 0.79397584 0.76704022 0.44937824 0.56998421 0.711116
  0.596789   0.46649991 0.5432462  0.56409088 0.78335748 0.81634548
  0.81777509 0.63996204 0.62322923 0.69410763 0.76594533 0.90961717
  0.92972588 0.9697281  1.06956724 0.96764675 0.

In [50]:
ind=7
print(np.dot(transition_matrices[ind,:x,:x],X_preds_KF[0,ind-1,:],)+ np.dot(transition_matrices[ind,:x,x:],X_preds_KF[i,ind-2,:]))
print(X_testD[0,ind,:])
print(X_preds_KF[0,ind,:])
print(X_cov_KF[0,ind,:,:])

[0.03906621 0.16819081 0.41206468 0.65850614 0.81200167 0.90162695
 0.95852095 0.95142463 0.99576739 0.99652995 0.90166995 0.78126225
 0.74531598 0.76157157 0.75272441 0.7045692  0.65124916 0.75605839
 0.86671898 0.87117564 0.8748494  0.87499001 0.86781526 0.90880739
 0.95113293 1.05568389 1.16143876 1.21426226 1.28255478 1.46948074]
tensor([0.0406, 0.1787, 0.4305, 0.6913, 0.7686, 0.8620, 0.9648, 0.9418, 1.0544,
        0.9829, 0.9351, 0.7801, 0.7466, 0.7435, 0.7862, 0.7483, 0.6325, 0.7509,
        0.8836, 0.9301, 0.8921, 0.9202, 0.8745, 0.8853, 0.9184, 1.0687, 1.2020,
        1.2237, 1.2586, 1.5655])
[0.03906621 0.16819081 0.41126791 0.65525195 0.80684962 0.89193812
 0.94169231 0.94570504 0.99893041 1.00502901 0.91206624 0.78761215
 0.74353927 0.75371747 0.74838193 0.70468186 0.64594138 0.74947619
 0.87240605 0.89243173 0.89771185 0.89951246 0.88407891 0.91236061
 0.95158568 1.05902909 1.17008451 1.23354112 1.3090492  1.48980223]
[[ 2.01230998e-06  1.56038803e-07 -8.33374532e-07  2.54

In [51]:
scores = np.zeros((D.shape[0],t))

for i in range(D.shape[0]):
    for j in range(t):
        scores[i,j] = logarithmic_scores(X_testD[i,j,:],X_preds_KF[i,j,:],X_cov_KF[i,j,:,:])/30
        


In [52]:
print(np.var(scores,axis=0))
print(np.mean(scores,axis=0))

[0.0171448  0.01712963 0.01576625 0.01646949 0.01719721 0.01725664
 0.0152329  0.02278364 0.02384825 0.02986514]
[-5.47408295 -5.48607676 -5.47774226 -5.49820548 -5.48545542 -5.47706699
 -5.49469335 -2.10017863 -1.90395404 -1.82889576]


### Best Kalman model with multiplicative noise

In [57]:
X_preds_KF = np.zeros(X_test_obsD.shape)
X_preds_KF[:,0:2,:] = X_test_obsD[:,0:2,:]
X_cov_KF = np.zeros((X_test_obsD.shape[0],t,x,x))
X_cov_KF[:,0,:] = Obs_noise**2*np.identity(x)
X_cov_KF[:,1,:] = Obs_noise**2*np.identity(x)


for i in range(X_test_obsD.shape[0]): 
    for j in range(t-2):        
        if j<(t-5):
            x1 = X_preds_KF[i,j,:]
            x2 = X_preds_KF[i,j+1,:]

            Initial_state = np.zeros((2*x))
            Initial_state[:x] = x2
            Initial_state[x:] = x1
            transition_covariance = np.zeros((2*x,2*x))
            Y=np.roll(X_testD[i,j+1,:],1)
            Y[0] =0
            transition_covariance[:x,:x] = Model_noise**2*np.diagflat(np.abs(X_testD[i,j+1,:]+Y))
            k = KalmanFilter(
    transition_matrices=transition_matrices[j,:,:],
             observation_matrices=observation_matrix, 
             transition_covariance=transition_covariance,
             observation_covariance=observation_covariance, 
             transition_offsets=None, 
             observation_offsets=None, 
             initial_state_mean=Initial_state, 
             initial_state_covariance=Initial_covariance, 
             random_state=None, 
             em_vars=['transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance'], 
             n_dim_state=None, 
             n_dim_obs=None
)
    
            X_Filter = np.zeros((2,2*x))
            X_Filter[0,:x] = X_test_obsD[i,j+1,:]
            X_Filter[0,x:] = X_test_obsD[i,j,:]
            X_Filter[1,:x] = X_test_obsD[i,j+2,:]
            X_Filter[1,x:] = X_test_obsD[i,j+1,:]
            KF = k.filter(X_Filter)
            X_preds_KF[i,j+2,:] = KF[0][1,:x]
            X_cov_KF[i,j+2,:,:] = KF[1][1,:x,:x]
            
        else:
            transition_covariance = np.zeros((2*x,2*x))
            Y=np.roll(X_testD[i,j+1,:],1)
            Y[0] =0
            transition_covariance[:x,:x] = Model_noise**2*np.diagflat(np.abs(X_testD[i,j+1,:]+Y))
            KFb =  np.zeros((2*x,2*x))
            KFb[:x,:x] = X_cov_KF[i,j+1,:,:]
            KFb = np.dot(np.dot(transition_matrices[j,:,:],KFb),transition_matrices[j,:,:].T) +transition_covariance
            X_cov_KF[i,j+2,:,:] = KFb[:x,:x]
            X_preds_KF[i,j+2,:] = np.dot(transition_matrices[j,:x,:x],X_preds_KF[i,j+1,:]) + np.dot(transition_matrices[j,:x,x:],X_preds_KF[i,j,:])


In [58]:
ind=9
print(np.dot(transition_matrices[ind-2,:x,:x],X_preds_KF[0,ind-1,:],)+ np.dot(transition_matrices[ind-2,:x,x:],X_preds_KF[i,ind-2,:]))
print(X_testD[0,ind,:])
print(X_preds_KF[0,ind,:])
print(X_cov_KF[0,ind,:,:])

[0.01403682 0.0792327  0.23928466 0.47910832 0.71526815 0.88448923
 0.98751718 1.0349268  1.06078418 1.08198891 1.05837922 0.96887577
 0.87605507 0.83486692 0.83022641 0.81066532 0.76434964 0.76425022
 0.84360291 0.92637844 0.95777211 0.96299041 0.96175603 0.9699662
 1.00009538 1.06875892 1.17113506 1.26647243 1.3510998  1.50126948]
tensor([0.0155, 0.0855, 0.2448, 0.5046, 0.7332, 0.8292, 0.9035, 1.0160, 1.0966,
        1.0862, 1.0908, 1.0559, 0.9451, 0.7663, 0.8751, 0.8465, 0.8123, 0.7500,
        0.8309, 0.9142, 1.0277, 1.0913, 1.0265, 0.9196, 0.9661, 1.0213, 1.2341,
        1.3936, 1.5138, 1.5317])
[0.01403682 0.0792327  0.23887156 0.4769939  0.71163636 0.87789651
 0.97531225 1.0224317  1.05646325 1.08510624 1.06836422 0.98039857
 0.88055023 0.83088513 0.82265162 0.80731582 0.76361539 0.76133313
 0.84018995 0.93346903 0.97676923 0.98470805 0.98135475 0.98434487
 1.00819343 1.07332069 1.17503774 1.27891879 1.37302379 1.52567635]
[[ 1.89283901e-05  8.38866329e-06  1.52883402e-06  1.351

In [59]:
scores = np.zeros((D.shape[0],t))

for i in range(D.shape[0]):
    for j in range(t):
        scores[i,j] = logarithmic_scores(X_testD[i,j,:],X_preds_KF[i,j,:],X_cov_KF[i,j,:,:])/30
        if np.isnan(scores[i,j]):
            print(np.linalg.det(X_cov_KF[i,j,:,:]))
            print(X_cov_KF[i,j,:10,:10])
    
print(scores)

[[-5.49340754 -5.3196195  -5.30980599 ... -2.21537657 -1.73168586
  -1.58278526]
 [-5.48081034 -5.53244172 -5.46343636 ... -1.84519029 -1.58737177
  -1.59553001]
 [-5.56452653 -5.26713228 -5.41994826 ... -1.99904568 -1.58961795
  -1.40148493]
 ...
 [-5.39730918 -5.56542827 -5.49806954 ... -2.35382225 -1.81245875
  -2.0556658 ]
 [-5.56108932 -5.42089919 -5.55525513 ... -2.22613174 -2.02590764
  -2.12953235]
 [-5.31216983 -5.38396796 -5.48879968 ... -2.20976421 -1.84921491
  -2.024598  ]]


In [61]:
print(np.var(scores,axis=0))
print(np.mean(scores,axis=0))

[0.0171448  0.01712963 0.01570986 0.01659348 0.01719949 0.0173468
 0.01553684 0.08453916 0.07378706 0.08690276]
[-5.47408295 -5.48607676 -5.4778851  -5.49837673 -5.48514436 -5.47538405
 -5.49291536 -1.95480335 -1.75659757 -1.68187385]
